# Ex20

Um empreendedor pode aplicar seu dinheiro em 5 projetos de investimento, cujos dados são apresentados abaixo:


| Projeto | 0       | 1       | 2       | 3       |
| -- | --       | --      | --      | --       |
| I       | (100,00)|         | 170,00  |         |
| II      | (150,00)|         |         | 220,00  |
| III     |         | (100,00)|         | 150,00  |
| IV      | (50,00) | 80,00   |         |         |
| V       |         | (70,00) |         | 130,00  |

Formule um modelo que ajude o investidor a determinar os projetos que maximizam o seu capital ao final do ano 3, considerando que:

- O capital inicial disponível é de R$ 220.000,00 e não poderá ser tomado empréstimo;
- Todo capital disponível poderá ser aplicado na poupança que rende juros de 10% ao ano;
- Os projetos I e V são mutuamente exclusivos.








# **Modelagem Matemática 

## **Definição das Variáveis**
Sejam as variáveis de decisão:

- $ x_i $: variável binária indicando se o projeto $ i $ é escolhido ($1$) ou não ($0$), para $ i \in \{I, II, III, IV, V\} $.
- $ S_t $: saldo disponível na poupança ao final do ano $ t $, para $ t \in \{0, 1, 2, 3\} $.



## **Função Objetivo**
Queremos **maximizar o capital ao final do ano 3**, ou seja:

$$
\max S_3
$$



## **Restrições**
### **1. Restrição de orçamento inicial (ano 0)**
O capital inicial disponível de R$ 220.000,00 será distribuído entre os investimentos e a poupança:

$$
C_I x_I + C_{II} x_{II} + C_{IV} x_{IV} + S_0 = 220
$$

onde:
- $ C_i $ representa o custo inicial de cada investimento $ i $.



### **2. Evolução do saldo na poupança**
O saldo da poupança evolui a cada ano acumulando juros e fluxos de caixa dos projetos escolhidos.

#### **Ano 1:**
$$
S_1 + C_{III} x_{III} + C_{V} x_{V} = S_0 \cdot (1.1) + R_{IV} x_{IV}
$$

#### **Ano 2:**
$$
S_2 = S_1 \cdot (1.1) + R_{I} x_{I}
$$

#### **Ano 3:**
$$
S_3 = S_2 \cdot (1.1) + R_{II} x_{II} + R_{III} x_{III} + R_{V} x_{V}
$$

onde:
- $ C_i $ representa o custo inicial dos projetos.
- $ R_i $ representa o retorno do investimento $ i $.
- A taxa de juros da poupança é **10% ao ano** (representada como **$1.1$**).



### **3. Mutual exclusividade entre os projetos I e V**
Os projetos **I** e **V** não podem ser escolhidos ao mesmo tempo:

$$
x_I + x_V \leq 1
$$



### **4. Variáveis binárias**
Os investimentos são **escolhidos ou não**, ou seja:

$$
x_i \in \{0,1\}, \quad \forall i \in \{I, II, III, IV, V\}
$$




In [20]:
import pyomo.environ as pyo

In [21]:
# Criação do modelo
model = pyo.ConcreteModel()

In [22]:

# Conjuntos
model.I = pyo.Set(initialize=["I", "II", "III", "IV", "V"])
model.T = pyo.RangeSet(0, 3) 


In [23]:
# Capital inicial disponível (R$ mil)
capital_inicial = 220

# Taxa de juros da poupança (10% ao ano)
taxa_juros = 1.1

# Fluxo de caixa dos projetos (R$ mil)
custo_inv = { "I":100, "II":150, "III":100, "IV":50, "V":70 }
retorno_in = { "I":170, "II":220, "III":150, "IV":80, "V":130 }
model.custo_inv = pyo.Param(model.I, initialize=custo_inv, default=0)
model.retorno_in = pyo.Param(model.I, initialize=retorno_in, default=0)


In [24]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.I, domain=pyo.Binary)
model.S = pyo.Var(model.T, domain=pyo.NonNegativeReals)

In [25]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return model.S[3]*taxa_juros

model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [26]:
# ------------------------------
# Restrições
# ------------------------------
# 1. Restrição de orçamento inicial (ano 0)
def restricao_orcamento(model):

    return model.custo_inv['I']*model.x['I'] +\
          model.custo_inv['II']*model.x['II']+\
          model.custo_inv['IV']*model.x['IV'] \
            + model.S[0] == capital_inicial

model.restricao_orcamento = pyo.Constraint(rule=restricao_orcamento)

# 2. Evolução do saldo na poupança 

def evolucao_saldo_ano_1(model):
    return model.S[1] + model.x['III'] * model.custo_inv['III'] + model.x['V']*model.custo_inv['V'] == \
    model.S[0]*taxa_juros+model.x['IV']*model.retorno_in['IV'] 


model.evolucao_saldo_ano_1 = pyo.Constraint(rule=evolucao_saldo_ano_1)

def evolucao_saldo_ano_2(model):
    return model.S[2] == \
    model.S[1]*taxa_juros+model.x['I']*model.retorno_in['I'] 

model.evolucao_saldo_ano_2 = pyo.Constraint(rule=evolucao_saldo_ano_2)

def evolucao_saldo_ano_3(model):
    return model.S[3] == \
    model.S[2]*taxa_juros+model.x['II']*model.retorno_in['II'] +model.x['III']*model.retorno_in['III'] + model.x['V']*model.retorno_in['V']

model.evolucao_saldo_ano_3 = pyo.Constraint(rule=evolucao_saldo_ano_3)
# 3. Mutual exclusividade entre os projetos I e V
def exclusividade(model):
    return model.x["I"] + model.x["V"] <= 1

model.exclusividade = pyo.Constraint(rule=exclusividade)


In [27]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex20.lp", io_options={"symbolic_solver_labels": True})

('ex20.lp', 2379947254160)

In [28]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 5 rows; 9 cols; 19 nonzeros; 5 integer variables (5 binary)
Coefficient ranges:
  Matrix [1e+00, 2e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+02]
Presolving model
4 rows, 8 cols, 14 nonzeros  0s
3 rows, 7 cols, 11 nonzeros  0s
Objective function is integral with scale 90.9091

Solving MIP model with:
   3 rows
   7 cols (5 binary, 0 integer, 2 implied int., 0 continuous)
   11 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         

In [29]:
print("Projetos escolhidos para investimento:")
for i in model.I:
    if model.x[i].value > 0.5:
        print(f"Projeto {i}")

print("\nSaldo na poupança ao final de cada ano:")
for t in model.T:
    print(f" ano {t}: R$ {model.S[t].value * 1000:.2f}")

print("\nSaldo final na poupança ao fim do ano 3:")
print(f" R$ {model.S[3].value * 1000:.2f}")

Projetos escolhidos para investimento:
Projeto I
Projeto III
Projeto IV

Saldo na poupança ao final de cada ano:
 ano 0: R$ 70000.00
 ano 1: R$ 57000.00
 ano 2: R$ 232700.00
 ano 3: R$ 405970.00

Saldo final na poupança ao fim do ano 3:
 R$ 405970.00
